In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
RL = 'https://www.case.org/conferences-training'
driver.get(RL)
try:
    driver.maximize_window()
except:
    pass

In [4]:
soup=bs(driver.page_source,'lxml')
cards=soup.find_all('div',{'class':'view__row'})
lastpage=' '.join(soup.find_all('li',{'class':'pager__item'})[-3].text.split()).split()[1]
len(cards),lastpage

(10, '3')

In [5]:
case=[]
n=0
brek=True
while brek:
    
    url=f'https://www.case.org/conferences-training?field_dates_end_value=1&field_recorded=All&keywords=&sort_by=field_dates&sort_order=&page={n}'
    driver.get(url)
    time.sleep(5)
    soup=bs(driver.page_source,'lxml')
    cards=soup.find_all('div',{'class':'view__row'})
    for i in cards:
        ritle=' '.join(i.find('h2',{'class':'media-teaser__title'}).text.split())
        web=i.find('article',{'class':'media-teaser'}).find('a')['href']
        link='https://www.case.org'+web
        dat_dur=' '.join(i.find('div',{'class':'media-teaser__eyebrow'}).text.split())
        try:
            loc=' '.join(i.find('p',{'class':re.compile('event-meta text.+')}).find('span',{'class':'country'}).text.split()).replace('and','').strip()
        except:
            loc=''
        #*******************
        driver.get(link)
        time.sleep(1)
        soupn=bs(driver.page_source,'lxml')
        title=' '.join(soupn.find('h1',{'class':'page-header__title'}).text.split())
        shep=' '.join(soupn.find('div',{'class':'page-header__subtitle'}).text.split()).split('|')#[1].strip()
    #     try:
    #         dat=' '.join(soupn.find('span',{'class':'date-display-range'}).text.split())
    #         #dam=''
    #         dam=' '.join(soupn.find('div',{'class':'page-header__subtitle'}).text.split()).replace(dat,'').replace('|','').strip()
    #     except:
        if len(shep)==1:
            dat=shep[0].strip()
            dam=''
        elif len(shep)==2:
            dat=shep[1].strip()
            dam=shep[0].strip()
        else:
            dat=shep[-1].strip()
            dam=shep[-2].strip()

        #dam=' '.join(soupn.find('div',{'class':'page-header__subtitle'}).text.split())#.replace(dat,'').replace('|','').strip()

        if ':' in dam or 'AM' in dam or 'PM' in dam or '-' in dam:
            tim=dam.upper().strip()
        else:
            loc=dam
        try:
            tess=dam.split(',')[1]
        except:
            loc=''
        try:
            tes=dam.split(':')[1]
        except:
            tim=''
        if tim=='' or tim==' ':
            time_=''
        else:
            if '-' in tim:
                stt=tim.split('-')[0].strip()
                shu=tim.split('-')[1].split()
                z=shu[-1].strip()
                ent=tim.split('-')[1].replace(z,'').strip()

            else:
                stt=tim.split()[0].strip()
                z=tim.split()[-1].strip()
                ent=''
            time_st={
                    'type':'general',
                    'Start_time':stt,
                    'end_time':ent,
                    'timezone':z,
                    'days':'all'
                }

            time_=json.dumps([time_st])

        #**********DATE STARTS HERE************
        dat=dat.replace('–','-')
        if '(' in dat:
            dat=dat.split('(')[0].strip()
        if dat=='':
            start_date=end_date=''
        else:
            mad=dat
            #try:
    #             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
    #             mad=ma.group()

            if '-' in mad:
                st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            elif 'and'in mad:
                st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            elif '&' in mad:
                st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            else:
                st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            #print(date__,st,en)
            ####################
            if any(c.isalpha() for c in st)==False:
            #      print('leg')
                pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
                sapa=pa.group()
                start=st+' '+sapa
                end=en
            #                        02        April      2022
            elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
             #   print('awujale')
                start=st
                end=en

            #*                        April       02      2022
            elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
            #       print('ala')
                pa=re.search('(\d{4})',st).group()
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()
                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',en).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba     

            #                      02     2022
            elif re.search('\d{1,2}\s+\d{4}',en):
            #      print('kum')
                pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()

                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',st).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba

            #                      02         April
            elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
            #        print('is')
                pa=re.search('(\d{4})',en)#, maxsplit=0)
                sapa=pa.group()
                start=st+' '+sapa
                end=en

            #*                      April           02
            elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
            #      print('bad')
                pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()

                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',en).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba

            else:
            #     print('shik')
                start=end=''
            if start=='':
                start_date=end_date=''
            else:
                pick=[start, end]
                spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
                date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
                start_date=date_[0]
                end_date=date_[1]
        event_info=' '.join(soupn.find('div',{'class':'introduction'}).text.split())
        try:
            fp=' '.join(soupn.find('div',{'class':'featured-person__content'}).find('h4',{'class':'featured-person__name'}).text.split())
            fpt=' '.join(soupn.find('div',{'class':'featured-person__title'}).text.split())
            #spk=[fp,fpt,'']
            speak={
                    'name':fp,
                    'title':fpt,
                    'link':''

            }
        except:
            speak=''
        if speak=='' or speak==[]:
            Speakerlist=''
        else:
            Speakerlist=json.dumps([speak])
        try:
            trs=soupn.find('div',{'class':'sponsor-group'}).find('a')['href']
            sps=trs.split('.')[-2]
        except:
            sps=''
        if sps=='' or sps==' ':
            sponsors=''
        else:
            sponsors=json.dumps([sps])


        def gu(luc):
            google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            google_map_url=driver.current_url
    #                 print(google_map_url)
            return(google_map_url)
            ######################################
        def get_google_map_url(location):
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


        def countr(locale):
            try:
                google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                try:
                    google_map_url=driver.find_element_by_id('lu_map').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                    except:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                time.sleep(1)
                try:
                    country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
                except:
                    country=driver.find_element_by_class_name('bwoZTb').text
    #                 print(country)
                return(country)
            except Exception as e:
                print(locale, ";country didn't go through")
                print(e)
                return("")
        ticket_list=''
        org_name='Council for Advancement and Support of Education'
        org_web='https://www.case.org/'
        org_pro='The Council for Advancement and Support of Education is the global nonprofit association dedicated to educational advancement professionals—in alumni relations, communications, development, marketing, and advancement services—who share the goal of championing education to transform lives and society.'
        logo=''    
        #sponsor=''
        agendalist=''
        type_=''
        category=''
        #Speakerlist='' 
        #time_=''
        event_web=link
        mail_=json.dumps(['europe@case.org'])
        if loc=='' or loc==' ' or 'Onli' in dam or 'Virt' in dam:
            on_off='1'
            city=''
            venue=''
            country=''
            googlePlaceUrl=''
        else:
            on_off='0'
            city=loc
            country=countr(city)
            venue=loc
            googlePlaceUrl=get_google_map_url(venue)
        
            
            
        case.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsors,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])
        del loc,tim
    n+=1
    if n==int(lastpage):
        brek=False
print('Done')

Ft. Lauderdale, FL ;country didn't go through
Message: target frame detached
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A5B8F3+2406643]
	Ordinal0 [0x009EAF31+1945393]
	Ordinal0 [0x008DC610+837136]
	Ordinal0 [0x008CD777+776055]
	Ordinal0 [0x008CC7C5+772037]
	Ordinal0 [0x008CCDC8+773576]
	Ordinal0 [0x008CCD58+773464]
	Ordinal0 [0x008D2F1A+798490]
	Ordinal0 [0x008CDF9B+778139]
	Ordinal0 [0x008CE4C5+779461]
	Ordinal0 [0x008CE2AF+778927]
	Ordinal0 [0x008CD8A6+776358]
	Ordinal0 [0x008CD11C+774428]
	Ordinal0 [0x008CCFB9+774073]
	Ordinal0 [0x008DDB00+842496]
	Ordinal0 [0x00933F4B+1195851]
	Ordinal0 [0x00924096+1130646]
	Ordinal0 [0x008FE636+976438]
	Ordinal0 [0x008FF546+980294]
	GetHandleVerifier [0x00CC9612+2498066]
	GetHandleVerifier [0x00CBC920+2445600]
	GetHandleVerifier [0x00AF4F2A+579370]
	GetHandleVerifier [0x00AF3D36+574774]
	Ordinal0 [0x009F1C0B+1973259]
	Ordinal0 [0x009F6688+1992328]
	Ordinal0 [0x009F6775+1992565]
	Ordinal0 [0x009FF8D1+2029777]
	Base

In [6]:
len(case)

28

In [7]:
case

[['https://www.case.org/conferences-training/diplomado-de-vinculacion-con-egresados',
  'Diplomado De Vinculación Con Egresados',
  '2022-01-24',
  '2022-01-29',
  '',
  '',
  '',
  'The Council for Advancement and Support of Education is the global nonprofit association dedicated to educational advancement professionals—in alumni relations, communications, development, marketing, and advancement services—who share the goal of championing education to transform lives and society.',
  'Council for Advancement and Support of Education',
  'https://www.case.org/',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'https://www.case.org/conferences-training/diplomado-de-vinculacion-con-egresados',
  '',
  '["europe@case.org"]',
  '',
  '1'],
 ['https://www.case.org/conferences-training/development-deans-and-academic-leaders-recording-series',
  'Development for Deans and Academic Leaders Recording Series',
  '2022-02-22',
  '2022-02-15',
  '',
  'Driven by CASE’s continuing efforts to devel

In [8]:
case_dict=case


case_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=case_dict)
case_df.to_csv("case.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [9]:
# truce=[]
# for k in range(1,29):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(1)
#     soupn=bs(driver.page_source,'lxml')
#     title=' '.join(soupn.find('h1',{'class':'page-header__title'}).text.split())
#     shep=' '.join(soupn.find('div',{'class':'page-header__subtitle'}).text.split()).split('|')#[1].strip()
# #     try:
# #         dat=' '.join(soupn.find('span',{'class':'date-display-range'}).text.split())
# #         #dam=''
# #         dam=' '.join(soupn.find('div',{'class':'page-header__subtitle'}).text.split()).replace(dat,'').replace('|','').strip()
# #     except:
#     if len(shep)==1:
#         dat=shep[0].strip()
#         dam=''
#     elif len(shep)==2:
#         dat=shep[1].strip()
#         dam=shep[0].strip()
#     else:
#         dat=shep[-1].strip()
#         dam=shep[-2].strip()
        
#     #dam=' '.join(soupn.find('div',{'class':'page-header__subtitle'}).text.split())#.replace(dat,'').replace('|','').strip()

#     if ':' in dam or 'AM' in dam or 'PM' in dam or '-' in dam:
#         tim=dam.upper().strip()
#     else:
#         loc=dam
#     try:
#         tess=dam.split(',')[1]
#     except:
#         loc=''
#     try:
#         tes=dam.split(':')[1]
#     except:
#         tim=''
#     if tim=='' or tim==' ':
#         time_=''
#     else:
#         if '-' in tim:
#             stt=tim.split('-')[0].strip()
#             shu=tim.split('-')[1].split()
#             z=shu[-1].strip()
#             ent=tim.split('-')[1].replace(z,'').strip()
            
#         else:
#             stt=tim.split()[0].strip()
#             z=tim.split()[-1].strip()
#             ent=''
#         time_st={
#                 'type':'general'
#                 'Start_time':stt,
#                 'end_time':ent,
#                 'timezone':z,
#                 'days':'all'
#             }
        
#         time_=json.dumps([time_st])

#     #**********DATE STARTS HERE************
#     dat=dat.replace('–','-')
#     if '(' in dat:
#         dat=dat.split('(')[0].strip()
#     if dat=='':
#         start_date=end_date=''
#     else:
#         mad=dat
#         #try:
# #             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
# #             mad=ma.group()

#         if '-' in mad:
#             st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#         elif 'and'in mad:
#             st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#         elif '&' in mad:
#             st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#         else:
#             st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         #print(date__,st,en)
#         ####################
#         if any(c.isalpha() for c in st)==False:
#         #      print('leg')
#             pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
#             sapa=pa.group()
#             start=st+' '+sapa
#             end=en
#         #                        02        April      2022
#         elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
#          #   print('awujale')
#             start=st
#             end=en

#         #*                        April       02      2022
#         elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
#         #       print('ala')
#             pa=re.search('(\d{4})',st).group()
#             sa=re.search('([A-Sa-z]{3,9})',st).group()
#             ta=re.search('(\d{1,2})',st).group()
#             start=ta+' '+sa+' '+pa
#             #
#             ba=re.search('(\d{4})',en).group()
#             ca=re.search('([A-Sa-z]{3,9})',en).group()
#             da=re.search('(\d{1,2})',en).group()
#             end=da+' '+ca+' '+ba     

#         #                      02     2022
#         elif re.search('\d{1,2}\s+\d{4}',en):
#         #      print('kum')
#             pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#             sa=re.search('([A-Sa-z]{3,9})',st).group()
#             ta=re.search('(\d{1,2})',st).group()

#             start=ta+' '+sa+' '+pa
#             #
#             ba=re.search('(\d{4})',en).group()
#             ca=re.search('([A-Sa-z]{3,9})',st).group()
#             da=re.search('(\d{1,2})',en).group()
#             end=da+' '+ca+' '+ba

#         #                      02         April
#         elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
#         #        print('is')
#             pa=re.search('(\d{4})',en)#, maxsplit=0)
#             sapa=pa.group()
#             start=st+' '+sapa
#             end=en

#         #*                      April           02
#         elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
#         #      print('bad')
#             pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#             sa=re.search('([A-Sa-z]{3,9})',st).group()
#             ta=re.search('(\d{1,2})',st).group()

#             start=ta+' '+sa+' '+pa
#             #
#             ba=re.search('(\d{4})',en).group()
#             ca=re.search('([A-Sa-z]{3,9})',en).group()
#             da=re.search('(\d{1,2})',en).group()
#             end=da+' '+ca+' '+ba

#         else:
#         #     print('shik')
#             start=end=''
#         if start=='':
#             start_date=end_date=''
#         else:
#             pick=[start, end]
#             spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
#             date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
#             start_date=date_[0]
#             end_date=date_[1]
#     info=' '.join(soupn.find('div',{'class':'introduction'}).text.split())
#     try:
#         fp=' '.join(soupn.find('div',{'class':'featured-person__content'}).find('h4',{'class':'featured-person__name'}).text.split())
#         fpt=' '.join(soupn.find('div',{'class':'featured-person__title'}).text.split())
#         #spk=[fp,fpt,'']
#         speak={
#                 'name':fp,
#                 'title':fpt,
#                 'link':''
            
#         }
#     except:
#         speak=''
#     if speak=='' or speak==[]:
#         Speakerlist=''
#     else:
#         Speakerlist=json.dumps([speak])
#     try:
#         trs=soupn.find('div',{'class':'sponsor-group'}).find('a')['href']
#         sps=trs.split('.')[-2]
#     except:
#         sps=''
#     if sps=='' or sps==' ':
#         sponsors=''
#     else:
#         sponsors=json.dumps([sps])
        
        
#     def gu(luc):
#         google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
#         time.sleep(randint(0,3))
#         driver.get(google_url_for_location)
#         time.sleep(4)
#         try:
#             google_map_url=driver.find_element_by_id('lu_map').click()
#         except:
#             try:
#                 google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
#             except:
#                 google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
#         time.sleep(1)
#         google_map_url=driver.current_url
# #                 print(google_map_url)
#         return(google_map_url)
#         ######################################
#     def get_google_map_url(location):
#         try:
#             return(gu(location))
#         except:
#             sha=location.split(',')
#             try:
#                 gu(sha[-3])
#             except:
#                 try:
#                     gu(sha[-2])
#                 except:
#                     try:
#                         gu(sha[-1])
#                     except Exception as e:
#                         print(location, "; url didn't go through")
#                         print(e)
#                         return("")


#     def countr(locale):
#         try:
#             google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
#             time.sleep(randint(0,3))
#             driver.get(google_url_for_location)
#             time.sleep(4)
#             try:
#                 google_map_url=driver.find_element_by_id('lu_map').click()
#             except:
#                 try:
#                     google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
#                 except:
#                     google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
#             time.sleep(1)
#             try:
#                 country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
#             except:
#                 country=driver.find_element_by_class_name('bwoZTb').text
# #                 print(country)
#             return(country)
#         except Exception as e:
#             print(locale, ";country didn't go through")
#             print(e)
#             return("")
        
        
#     if loc=='' or loc==' ':
#         on_off='1'
#         city=''
#         venue=''
#         country=''
#         googlePlaceUrl=''
#     else:
#         on_off='0'
#         city=loc
#         country=countr(city)
#         venue=loc
#         googlePlaceUrl=get_google_map_url(venue)
        
        
        
        
#     truce.append([title,loc,time_,start_date,sponsors,city,country,end_date,venue,info,googlePlaceUrl,on_off,Speakerlist])
#     del loc,tim
# truce